# Générateur de pages

Ce script Python complète le template «Trombinoscope» de la [Toolbox](https://labs.letemps.ch/toolbox/).

Il génère des pages HTML statiques en remplaçant le titre et la description pour les réseaux sociaux.

In [37]:
from bs4 import BeautifulSoup
import requests
import pandas as pd

In [38]:
df = pd.read_csv('data-for-generator.csv')

In [39]:
response = requests.get('https://labs.letemps.ch/interactive/2019/patrons-smi/')

In [40]:
df.head()

,Unnamed: 0,cardId,cardFilter5Display,cardTitle,cardSubtitle,datePrint,cardFilter3Display,stocksHolded,stocksComment,stocksPercentage,...,cardFilter1Display,cardFilter3Tri,Salary,cardFilter1Tri,evolution-action,cardFilter2Tri,cardSection1Text,cardLead,parameter,cardFilter3DisplayLong
0,0,0,ABB Ltd,Peter Voser,Il ne voulait plus être directeur,NaN,"1,2 million",201076.0,(pas de stock-options),"~0,56%",...,3 mois,f-salaire-1,f-salaire-1,f-experience-1,-7.59,f-negative,\n<p>Le départ n’avait visiblement pas été ant...,ABB s’est trouvé un nouveau directeur mi-août ...,peter-voser,"1,2 million de francs"
1,1,1,Adecco,Alain Dehaze,La tête de l’emploi,OK lundi,"2,5 million",43169.0,NaN,0.026%,...,3 ans,f-salaire-1,f-salaire-1,f-experience-2,-18.57,f-negative,\n<p>Classique. Vêtu d’un costume gris à carre...,"Le patron d’Adecco, citoyen belge qui s’est am...",alain-dehaze,"2,5 million de francs"
2,2,2,Credit Suisse Group AG,Tidjane Thiam,Bientôt Suisse,NaN,"12,7 millions",1055008.0,(64302 sans les actions non acquises),NaN,...,4 ans,f-salaire-3,f-salaire-3,f-experience-2,-54.98,f-negative,\n<p>Tidjane Thiam n’en veut visiblement pas à...,Arrivé à l’été 2015 pour redresser Credit Suis...,tidjane-thiam,"12,7 millions de francs"
3,3,3,Geberit,Christian Buhl,«Humble et pragmatique»,NaN,"1,47 millions",NaN,(call options: 78425),"<0,1%",...,4 ans,f-salaire-1,f-salaire-1,f-experience-2,34.75,f-positive,\n<p>C’est tout le paradoxe. Ses produits nous...,"Peu visible dans les médias, le patron de Gebe...",christian-buhl,"1,47 millions de francs"
4,4,4,Givaudan,Gilles Andrier,Mystérieux comme un parfum,NaN,"5,6 millions",7309.0,NaN,"0,079%",...,14 ans,f-salaire-2,f-salaire-2,f-experience-3,273.00,f-tres-positive,\n<p>Pour entrer en contact avec Gilles Andrie...,Le discret patron du fabricant d’arômes a fait...,gilles-andrier,"5,6 millions de francs"


In [41]:
soup = BeautifulSoup()

In [42]:
df['cardTitle']

0               Peter Voser
1              Alain Dehaze
2             Tidjane Thiam
3            Christian Buhl
4            Gilles Andrier
5            David Endicott
6               Jan Jenisch
7                 Marc Funk
8            Mark Schneider
9         Vasant Narasimhan
10           Jérôme Lambert
11           Severin Schwan
12               Frankie Ng
13             Paul Schuler
14               Nick Hayek
15             Urs Schaeppi
16            Patrick Frost
17    Christian Mumenthaler
18           Sergio Ermotti
19              Mario Greco
Name: cardTitle, dtype: object

In [43]:
for i, row in df.iterrows():
    if row['parameter'] != row['parameter']:
        continue

    doc = BeautifulSoup(response.content)
    
    # Si besoin ajouter <base href...>
    #base = doc.new_tag('base')
    #doc.head.insert(0, base)
    
    # Canonical URL
    canonical = 'https://labs.letemps.ch/interactive/2019/patrons-smi/portrait-{}.html'.format(row['parameter'])
    doc.select('link[rel="canonical"]')[0].attrs['href'] = canonical
    doc.select('meta[property="og:url"]')[0].attrs['content'] = canonical
    doc.select('meta[name="twitter:url"]')[0].attrs['content'] = canonical
        
    name = row['cardTitle']
    img_social = 'https://labs.letemps.ch/interactive/2019/patrons-smi/img/' + row['cardImage'] + '.jpg'
    
    print(name, row['cardImage'], img_social)
    title = doc.find('title')
    newtitle = doc.new_tag('title')
    newtitle.string = name + ' – ' + title.string
    #
    doc.find('title').replaceWith(newtitle)
    
    doc.select('meta[property="og:title"]')[0].attrs['content'] = newtitle.string
#    doc.select('meta[name="twitter:description"]')[0].attrs['content'] = newtitle.string
    
    doc.select('meta[property="og:image"]')[0].attrs['content'] = img_social
    doc.select('link[rel="image_src"]')[0].attrs['href'] = img_social
    doc.select('meta[name="twitter:image"]')[0].attrs['content'] = img_social
    
    with open("portrait/portrait-{}.html".format(row['parameter']), 'w') as fp:
        fp.write(str(doc))
    with open("/Users/rnp/Sites/patrons-smi/portrait-{}.html".format(row['parameter']), 'w') as fp:
        fp.write(str(doc))        

Peter Voser Voser https://labs.letemps.ch/interactive/2019/patrons-smi/img/Voser.jpg
Alain Dehaze Dehaze https://labs.letemps.ch/interactive/2019/patrons-smi/img/Dehaze.jpg
Tidjane Thiam Thiam https://labs.letemps.ch/interactive/2019/patrons-smi/img/Thiam.jpg
Christian Buhl Buhl https://labs.letemps.ch/interactive/2019/patrons-smi/img/Buhl.jpg
Gilles Andrier Andrier https://labs.letemps.ch/interactive/2019/patrons-smi/img/Andrier.jpg
David Endicott Endicott https://labs.letemps.ch/interactive/2019/patrons-smi/img/Endicott.jpg
Jan Jenisch Jenisch https://labs.letemps.ch/interactive/2019/patrons-smi/img/Jenisch.jpg
Marc Funk Funk https://labs.letemps.ch/interactive/2019/patrons-smi/img/Funk.jpg
Mark Schneider Schneider https://labs.letemps.ch/interactive/2019/patrons-smi/img/Schneider.jpg
Vasant Narasimhan Narasimhan https://labs.letemps.ch/interactive/2019/patrons-smi/img/Narasimhan.jpg
Jérôme Lambert Lambert https://labs.letemps.ch/interactive/2019/patrons-smi/img/Lambert.jpg
Severin S

In [44]:
!open portrait